In [12]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, WebSearchTool, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
import os
from pydantic import BaseModel, Field
import asyncio
from IPython.display import display, Markdown
from typing import Dict


In [13]:
load_dotenv(override=True)

True

In [14]:
ANTHROPIC_BASE_URL = "https://api.anthropic.com/v1/"

anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

anthropic_client = AsyncOpenAI(base_url=ANTHROPIC_BASE_URL, api_key=anthropic_api_key)

anthropic_model = OpenAIChatCompletionsModel(model="claude-3-7-sonnet-latest", openai_client=anthropic_client)


In [15]:
INSTRUCTIONS = "You are a researcher. You are doing research about quantum computing." \
"Given the search term, you search the web for the answer. Produce a concise and clear epplanation of the results." \
"Capture the main points and explain them in simple terms." \
"Add links to the sources you used." \
"Provide no more than 3 sources.";

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini"
)

In [16]:
NUMBER_OF_SEARCHES = 3

INSTRUCTIONS = f"You are a research assistant. Given a query, come up with a set of web searches \
to perform the best answer to the query. You can perform up to {NUMBER_OF_SEARCHES} searches. ";

print(INSTRUCTIONS)

class WebSearchItem(BaseModel):
    reason: str =  Field(description="The reason for why this search was performed.")
    query: str = Field(description="The search query used.")


class WebSearchOutput(BaseModel):
    search_items: list[WebSearchItem] = Field(description="List of search items with reasons and queries.")

planner_agent = Agent(
    name="Search planner agent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchOutput
)



You are a research assistant. Given a query, come up with a set of web searches to perform the best answer to the query. You can perform up to 3 searches. 


In [17]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "Using the provided summary of research findings, compose a detailed report that elaborates on the key points. "
    "Ensure the report is well-structured, clear, and provides in-depth insights into the research topic."
)
class ReportData(BaseModel):
    summary: str = Field(description="A concise summary of the research findings.")
    report: str = Field(description="A detailed report of the research findings.")

report_agent = Agent(
    name="Report agent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData
)

In [18]:
async def plan_search(query: str):
    """Use the planner agent to create a search plan."""
    print("Planning search...")
    result = await Runner.run(planner_agent, f"Query:  {query}")
    print(f"Will perform {len(result.final_output.search_items)} searches.")
    return result.final_output

In [19]:
async def execute_search_plan(search_plan: WebSearchOutput):
    """Execute the search plan using the search agent."""
    print("Executing search plan...")
    all_results = []
    for item in search_plan.search_items:
        print(f"Performing search for: {item.query} (Reason: {item.reason})")
        result = await Runner.run(search_agent, item.query)
        all_results.append(result.final_output)
    return all_results

In [20]:
async def summarize_search_results(search_result: str):
    """Summarize the search results using the search agent."""
    print("Summarizing search results...")
    summary_instructions = "Summarize the following search results into a concise explanation with main points and sources."
    result = await Runner.run(report_agent, f"{summary_instructions}\n\n{search_result}")
    return result.final_output

In [21]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    search_plan = await plan_search(query)
    search_results = await execute_search_plan(search_plan)
    print(f"Search results: {search_results}")
    summary = await summarize_search_results(search_results)

#display(Markdown(summary.final_output))

Planning search...
Will perform 3 searches.
Executing search plan...
Performing search for: latest AI agent frameworks 2025 (Reason: To gather information on emerging AI agent frameworks set to launch or achieve prominence in 2025.)
Performing search for: top AI agent frameworks 2025 features (Reason: To understand the features and capabilities of AI agent frameworks expected to be popular in 2025.)
Performing search for: AI agent technology trends 2025 analysis (Reason: To find expert opinions and analyses on trends in AI agent technology for 2025.)
Search results: ["As of November 2025, several advanced AI agent frameworks have emerged, each offering unique capabilities for developing intelligent, autonomous systems. Here are some notable frameworks:\n\n\n\n**OpenAI Agents SDK**  \nA lightweight Python framework released in March 2025, focusing on creating multi-agent workflows with comprehensive tracing and guardrails. It is compatible with over 100 different large language models (